# Demo: Pipeline Parallelism with DeepSpeed - The "Happy Path"

The purpose of this demo is to provide a clear, simple, and successful "first look" at pipeline parallelism. We will demonstrate:
1.  How a simple, sequential model can be **automatically partitioned** by DeepSpeed across multiple GPUs.
2.  The basic **mechanics** of launching a DeepSpeed job using a configuration file.
3.  The **outcome**: a single model running collaboratively on multiple devices.

This demo represents the ideal, "happy path" scenario. The model we use is structured perfectly for DeepSpeed's automatic partitioning, so we expect it to work right out of the box.

### How We'll Run This in a Notebook
The `deepspeed` command is an external launcher. To make this work seamlessly in a notebook, we will:
1.  **Programmatically create** a JSON configuration file.
2.  **Write our Python logic to a script** using the `%%writefile` magic command.
3.  **Execute the `deepspeed` launcher** on that script directly from the notebook using `!`.

## 1. Environment Setup

First, we'll install the necessary libraries and then import them, checking to ensure our environment is set up correctly. This demo requires at least 4 GPUs to see pipeline parallelism in action.

In [1]:
!pip install datasets rouge_score deepspeed transformers evaluate huggingface_hub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 108.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 175.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 194.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 179.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 129.9 MB/s eta 0:00:00
Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`),

In [ ]:
import torch
import torch.nn as nn
import deepspeed
import json
import os

print(f"PyTorch version: {torch.__version__}")
print(f"DeepSpeed version: {deepspeed.__version__}")
print(f"CUDA is available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    if torch.cuda.device_count() < 4:
        print("!! WARNING: This demo is designed for 4 GPUs. It may not run correctly. !!")

[2025-07-14 04:08:19,216] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[2025-07-14 04:08:33,980] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
PyTorch version: 2.2.2
DeepSpeed version: 0.17.2
CUDA is available: True
Number of GPUs available: 4


## 2. Define a "Pipeline-Friendly" Model

Next, we'll create a model that is perfectly suited for pipeline parallelism. It is a simple `nn.Sequential` model.

DeepSpeed's `"uniform"` partition method can easily inspect this sequence of layers and divide it into `N` even chunks for `N` GPUs. This makes the setup extremely simple.

In [ ]:
# We will write this model definition directly into our script in the next step.
# Here is the code for inspection:

class SequentialDemoModel(nn.Module):
    def __init__(self, hidden_size=1024, num_layers=8):
        super().__init__()

        # A simple list of layers
        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())

        # Wrapping layers in `nn.Sequential` is the key to making partitioning easy
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# Let's inspect the model structure
model_for_inspection = SequentialDemoModel()
print("Model Structure for the Demo:")
print(model_for_inspection)
print("\nThis nn.Sequential structure is ideal for DeepSpeed's automatic partitioning.")

Model Structure for the Demo:
SequentialDemoModel(
  (layers): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): ReLU()
    (8): Linear(in_features=1024, out_features=1024, bias=True)
    (9): ReLU()
    (10): Linear(in_features=1024, out_features=1024, bias=True)
    (11): ReLU()
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): ReLU()
    (14): Linear(in_features=1024, out_features=1024, bias=True)
    (15): ReLU()
  )
)

This nn.Sequential structure is ideal for DeepSpeed's automatic partitioning.


## 3. Create the DeepSpeed Configuration

This JSON configuration file is the control panel for DeepSpeed. It tells the launcher how to set up our job. The most important part for this demo is the `"pipeline"` section, where we define how the model should be split.

- `stages`: The number of pipeline stages to split the model into. This should match the number of GPUs we use.
- `partition_method`: How to partition the layers. `"uniform"` splits the layer sequence as evenly as possible.
- `micro_batch_size`: The size of the smaller data chunks that flow through the pipeline to keep all GPUs busy.

In [ ]:
ds_config_demo = {
  "train_batch_size": 16, # Global batch size

  # A dummy optimizer is required by the DeepSpeed initializer
  "optimizer": { "type": "SGD", "params": { "lr": 0.001 } },

  # Pipeline Parallelism Configuration
  "pipeline": {
    "stages": 2,              # We will split the model into 2 stages for our 2 GPUs
    "partition_method": "uniform", # Tells DeepSpeed to split layers evenly. Perfect for nn.Sequential.
    "micro_batch_size": 8     # We split our global batch size into smaller micro-batches
  },

  "comms_logger": {
    "enabled": False,
    "verbose": False,
    "debug": False
  }
}

# Write the configuration to a file
config_filename_demo = 'ds_config_demo.json'
with open(config_filename_demo, 'w') as f:
    json.dump(ds_config_demo, f, indent=2)

print(f"DeepSpeed configuration file '{config_filename_demo}' created.")
print("\n--- Demo Config Contents ---")
print(json.dumps(ds_config_demo, indent=2))
print("----------------------------")

DeepSpeed configuration file 'ds_config_demo.json' created.

--- Demo Config Contents ---
{
  "train_batch_size": 16,
  "optimizer": {
    "type": "SGD",
    "params": {
      "lr": 0.001
    }
  },
  "pipeline": {
    "stages": 2,
    "partition_method": "uniform",
    "micro_batch_size": 8
  },
  "comms_logger": {
    "enabled": false,
    "verbose": false,
    "debug": false
  }
}
----------------------------


## 4. Write the Main Execution Script

Now we'll package our logic into a Python script. The `deepspeed` launcher will execute this script on each GPU. 

Inside, the `deepspeed.initialize()` function is the key. 

It reads the configuration, performs the model partitioning, and returns a wrapped `model_engine` that handles all the complex distributed logic for us.

In [46]:
%%writefile demo_pipeline_script.py

import torch
import torch.nn as nn
import deepspeed
import argparse

# The model definition is included in the script
class SequentialDemoModel(nn.Module):
    def __init__(self, hidden_size=1024, num_layers=8):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def main():
    
    # Standard DeepSpeed argument parsing
    parser = argparse.ArgumentParser()
    parser.add_argument("--local_rank", type=int, default=-1, help="local rank")
    parser = deepspeed.add_config_arguments(parser)
    args = parser.parse_args()

    # 1. Instantiate the Model
    # DeepSpeed will handle placing it on the correct device.
    model = SequentialDemoModel()

    # 2. Initialize with DeepSpeed
    # This is the core function where the magic happens. DeepSpeed reads the
    # config, partitions the model, and wraps it in a `model_engine`.
    model_engine, _, _, _ = deepspeed.initialize(
        args=args,
        model=model,
        model_parameters=model.parameters(),
    )

    # 3. Print device information to verify partitioning
    # Each rank (GPU) will print which device its partition is on.
    print(
        f"Rank {model_engine.local_rank}: My model partition is on device: {model_engine.device}",
        flush=True
    )

    # 4. Run a forward pass
    # We create input data and pass it to the engine. DeepSpeed handles
    # passing the data through the pipeline stages automatically.
    batch_size = 16
    hidden_size = 1024
    dummy_input = torch.randn(batch_size, hidden_size, device=model_engine.device)

    output = model_engine(dummy_input)

    # The final output is only available on the last stage of the pipeline.
    # We can check this to confirm the run was successful.
    if model_engine.is_last_stage():
        print(f"\nRank {model_engine.local_rank} (Last Stage): Inference successful!", flush=True)
        print(f"Final output shape: {output.shape}", flush=True)

if __name__ == "__main__":
    main()

Overwriting demo_pipeline_script.py


## 5. Launch the DeepSpeed Job!

It's time to run our demo. We will execute the `deepspeed` launcher, telling it to use 4 GPUs. 

We'll also add some environment variables (`DS_LOG_LEVEL=ERROR` and `PYTHONWARNINGS=ignore`) to reduce log verbosity and keep the output clean.

Pay close attention to the output logs. You should see a message from each rank confirming which GPU its model partition is on.

In [ ]:
print("🚀 Launching DeepSpeed Pipeline Parallelism Demo...")
!DS_LOG_LEVEL=ERROR PYTHONWARNINGS=ignore deepspeed --num_gpus 4 demo_pipeline_script.py --deepspeed_config ds_config_demo.json

🚀 Launching DeepSpeed Pipeline Parallelism Demo...
Rank 1: My model partition is on device: cuda:1
Rank 2: My model partition is on device: cuda:2
Rank 3: My model partition is on device: cuda:3
Rank 0: My model partition is on device: cuda:0

Rank 3 (Last Stage): Inference successful!
Final output shape: torch.Size([16, 1024])

[2025-07-14 04:53:48,894] [INFO] [launch.py:351:main] Process 25565 exits successfully.
[2025-07-14 04:53:48,895] [INFO] [launch.py:351:main] Process 25564 exits successfully.
[2025-07-14 04:53:48,895] [INFO] [launch.py:351:main] Process 25567 exits successfully.
[2025-07-14 04:53:49,896] [INFO] [launch.py:351:main] Process 25566 exits successfully.


## 6. Analysis and Conclusion

You should have seen output similar to this:

```
Rank 0: My model partition is on device: cuda:0
Rank 1: My model partition is on device: cuda:1
...
Rank 3 (Last Stage): Inference successful!
Final output shape: torch.Size()
```

**Success!** This confirms that:
- DeepSpeed successfully launched 4 processes, one for each GPU.
- It partitioned our `SequentialDemoModel` and placed each partition on a separate GPU.
- It correctly managed the data flow through the pipeline to produce a final result on the last stage.
